# Lista 8

## PyTorch

(6pkt + 2pkt)

Na liście znajduje się 1 zadanie. Po rozwiązaniu go, pokaż kod prowadzącemu i odpowiedz na **pytanie kontrolne** — tylko wtedy przyznajemy punkty. Dodatkowo prześlij zadanie na platformie skos.

Dodatkowe zadanie oznaczone jest ⭐️ i jest warte 2pkt.

## Jeśli ćwiczenia będą się przedłuały...

Odpowiedz na ponisze pytania pisemnie i prześlij zadanie na skos. Do zobaczenia na kolejnych zajęciach! 😀

1. (W tym pytaniu nie ma złej odpowiedzi, jeśli umiesz ją uzasadnić) Zbiorem danych są obrazki, na których są napisane cyfry. Jakie transformacje moznaby zastosować na obrazkach, za pomocą których będziemy uczyli potem sieć?
2. Czy w sieci neuronowej dla ponizszego zbioru danych i dla tej konkretnej architektury potrzebne jest `Flatten`? Dlaczego?
3. W zeszłym tygodniu omawialiśmy Cross Validation. Jak wyglądałoby ono w przypadku uczenia sieci neuronowej?

## Wytrenuj sieć neuronową

Twoim celem jest przygotować kompletny pipeline do trenowania prostej sieci neuronowej w PyTorchu: od datasetu, przez dataloadery i model, aż po trening, walidację i test.

---

### Dataset i transformacje

Przygotuj dane wejściowe korzystając z `torchvision.datasets.MNIST`.

#### Kroki:

1. Zaimportuj konieczne biblioteki:

   * `torch`
   * `torchvision`
   * `torch.utils.data`
   * oraz moduł transformacji: `torchvision.transforms as transforms`

2. Pobierz dataset **MNIST** dla:

   * zbioru treningowego,
   * zbioru testowego.

3. Zastosuj transformacje:

   * **obowiązkowo:** `transforms.ToTensor()`
   * **opcjonalnie:** normalizacja
     `transforms.Normalize((0.5,), (0.5,))`

4. Podziel zbiór treningowy na dwie części przy użyciu `torch.utils.data.random_split`:

   * **90%** — train
   * **10%** — validation

5. **Nie twórz własnego datasetu.** Użyj wbudowanego `torchvision.datasets.MNIST`.

**Uwaga:** Upewnij się, że rozmiar batcha to `(batch, 1, 28, 28)` — transformacje nie mogą zmieniać tego formatu.

---

### Dataloadery

Utwórz trzy `DataLoader`-y:

* **train** — `shuffle=True`
* **validation** — `shuffle=False`
* **test** — `shuffle=False`

Z parametrami:

* `batch_size = 64`
* `num_workers = 0`
  (żeby uniknąć problemów np. na Windowsowych laptopach)

---

### Sieć neuronowa (MLP)

Zaimplementuj model dziedzicząc po `nn.Module`.

#### Architektura — dokładnie w tej kolejności:

1. `Flatten()`
2. `Linear(28*28, 256)`
3. `ReLU()`
4. `Linear(256, 128)`
5. `ReLU()`
6. `Linear(128, 10)`

**Wskazówki:**

* Warstwy umieść w `nn.Sequential`.
* Domyślna aktywacja to ReLU (ale możesz eksperymentować).

#### Dodatkowe elementy:

1. Funkcja straty: **CrossEntropyLoss**.
2. Optymalizator: **Adam**, `lr=0.001`.

---

### Pętla treningowa

Napisz pętlę uczącą, która dla każdej epoki:

#### W trybie `model.train()`:

Dla każdego batcha:

* oblicza logitsy (predykcje),
* oblicza stratę,
* wykonuje:

  * `optimizer.zero_grad()`
  * `loss.backward()`
  * `optimizer.step()`
* liczy poprawne odpowiedzi.

#### Po epce — wyświetl:

* średni **train loss**
* **train accuracy**

**Wskazówka (accuracy):**

```python
pred = logits.argmax(dim=1)
correct += (pred == y).sum().item()
```

---

### Walidacja + zapis najlepszego modelu

Dodaj funkcję walidującą:

* `model.eval()`
* `torch.no_grad()`

Policz:

* średni **val loss**
* **val accuracy**

#### Po każdej epoce:

* sprawdź, czy `val_loss` jest najlepszy w historii,
* jeśli tak — zapisz model:

```python
torch.save(model.state_dict(), "best_model.pth")
```

* wypisz komunikat:

> **"Zapisano nowy najlepszy model!"**

---

### Testowanie najlepszego modelu

1. Wczytaj najlepsze wagi:

   ```python
   model.load_state_dict(torch.load("best_model.pth"))
   ```

2. Przeprowadź ewaluację na zbiorze **testowym** (procedura taka jak przy walidacji).

3. Wyświetl **finalną test accuracy**.

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

# Dataset + Transformacje

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_full = datasets.MNIST(root=".", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root=".", train=False, transform=transform, download=True)

# Podział train/val
train_size = int(0.9 * len(train_full))
val_size = len(train_full) - train_size
train_dataset, val_dataset = random_split(train_full, [train_size, val_size])

# DataLoadery

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=0)
# Model

class SimpleMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = SimpleMLP()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Funkcje treningowe i walidacyjne

def train_epoch(model, loader):
    model.train()
    total_loss, correct = 0, 0

    size = len(loader.dataset)
    num_batches = len(loader)

    for x, y in loader:
        outputs = model(x)
        loss = criterion(outputs, y)

        total_loss += loss.item()
        correct += (outputs.argmax(dim=1) == y).type(torch.float).sum().item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    avg_loss = total_loss / num_batches
    accuracy = correct / size

    return avg_loss, accuracy

def eval_epoch(model, loader):
    total_loss, correct = 0, 0

    size = len(loader.dataset)
    num_batches = len(loader)

    model.eval()

    with torch.no_grad():
        for x, y in loader:
            pred = model(x)
            total_loss += criterion(pred, y).item()
            correct += (pred.argmax(dim=1) == y).sum().item()

    avg_loss = total_loss / num_batches
    accuracy = correct / size

    print(f"Test Error: \n Accuracy: {(100*accuracy):>0.1f}%, Avg loss: {avg_loss:>8f} \n")

    return avg_loss, accuracy

# Trening z zapisem najlepszego modelu

EPOCHS = 3
best_val_loss = float('inf')

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader)
    val_loss, val_acc = eval_epoch(model, val_loader)

    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  Train loss: {train_loss:.4f}, acc: {train_acc:.4f}")
    print(f"  Val loss:   {val_loss:.4f}, acc: {val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), "best_model.pth")
        print("Zapisano nowy najlepszy model!")

# Test końcowy: wczytanie najlepszego modelu

model.load_state_dict(torch.load("best_model.pth"))

test_loss, test_acc = eval_epoch(model, test_loader)
print("\nFinal test accuracy:", test_acc)


Test Error: 
 Accuracy: 94.0%, Avg loss: 0.188643 

Epoch 1/3
  Train loss: 0.3612, acc: 0.8913
  Val loss:   0.1886, acc: 0.9395
Zapisano nowy najlepszy model!
Test Error: 
 Accuracy: 95.7%, Avg loss: 0.129200 

Epoch 2/3
  Train loss: 0.1669, acc: 0.9493
  Val loss:   0.1292, acc: 0.9572
Zapisano nowy najlepszy model!
Test Error: 
 Accuracy: 96.2%, Avg loss: 0.111807 

Epoch 3/3
  Train loss: 0.1193, acc: 0.9626
  Val loss:   0.1118, acc: 0.9615
Zapisano nowy najlepszy model!
Test Error: 
 Accuracy: 96.4%, Avg loss: 0.114338 


Final test accuracy: 0.9642


## Jeśli ćwiczenia będą się przedłuały...

Odpowiedz na ponisze pytania pisemnie i prześlij zadanie na skos. Do zobaczenia na kolejnych zajęciach! 😀

1. (W tym pytaniu nie ma złej odpowiedzi, jeśli umiesz ją uzasadnić) Zbiorem danych są obrazki, na których są napisane cyfry. Jakie transformacje moznaby zastosować na obrazkach, za pomocą których będziemy uczyli potem sieć?
2. Czy w sieci neuronowej dla ponizszego zbioru danych i dla tej konkretnej architektury potrzebne jest `Flatten`? Dlaczego?
3. W zeszłym tygodniu omawialiśmy Cross Validation. Jak wyglądałoby ono w przypadku uczenia sieci neuronowej?`

1.

lekkie obroty, zmiany kolorów, lekkie rozciąganie, blur  
nie powinniśmy stosować odbić lustrzanych, etc - nie chcemy tak zniekształcić obrazu, że nieprzypomina on już liczyb


2.

potrzebne, bo sieć przyjmuje jako input tensor jednowymiarowy (wektor)

3.

analogicznie - dzielimy zbiór treningowy na kilka grup i trenujemy sieć biorąc jedną z grup jako walidacja a resztę jako trening

## ⭐️ KMNIST — Kompletny Pipeline od Surowych Danych do Trenowania Sieci

W tym zadaniu pracujesz z datasetem **KMNIST**, ale **bez używania gotowego datasetu z `torchvision`**.

Zrobisz wszystko **ręcznie**:

* pobierzesz dane z `torchvision` (zostało zrobione za Ciebie),
* zapiszesz pojedyncze obrazki jako `.jpg` (zostało zrobione za Ciebie),
* zapiszesz etykiety do plików `.csv` (zostało zrobione za Ciebie),
* zbudujesz własną klasę `Dataset`,
* przygotujesz dataloadery,
* stworzysz i wytrenujesz sieć neuronową opartą o CNN.

---

### Przygotowanie danych

#### Pobranie KMNIST

Zostało zrobione za Ciebie

---

### Twoja własna klasa Dataset

Stwórz klasę dziedziczącą po `torch.utils.data.Dataset`.

#### Wymagania:

#### `__init__`:

* wczytuje odpowiedni plik CSV,
* przechowuje listę ścieżek do obrazów i odpowiadających im etykiet.

#### `__getitem__`:

* otwiera obraz `PIL`,
* stosuje transformacje,
* zwraca:

  ```python
  (tensor, label)
  ```

#### `__len__`:

* zwraca liczbę przykładów.

#### Transformacje

**Obowiązkowe:**

* `transforms.ToTensor()`

**Opcjonalne:**

* `transforms.Normalize((0.5,), (0.5,))`

---

### Podział danych na train / val / test

Dla danych treningowych:

* **90%** → train
* **10%** → validation

Użyj:

```python
torch.utils.data.random_split
```

---

### Dataloadery

Utwórz trzy DataLoadery:

* **train** — `shuffle=True`
* **val** — `shuffle=False`
* **test** — `shuffle=False`

#### Parametry:

* `batch_size = 64`
* `num_workers = 0`

---

### Sieć neuronowa — CNN

Zaimplementuj prosty CNN w PyTorch. Jeśli będziesz miał problem z implementacją napisz maila do prowadzącego - pomożemy.

#### Zalecana architektura:

1. `Conv2d(1, 32, 3, padding=1)`
2. `ReLU()`
3. `MaxPool2d(2)`
4. `Conv2d(32, 64, 3, padding=1)`
5. `ReLU()`
6. `MaxPool2d(2)`
7. `Flatten()`
8. `Linear(64*7*7, 128)`
9. `ReLU()`
10. `Linear(128, 10)`

#### Wskazówki:

* Możesz użyć `nn.Sequential` albo napisać `forward()` ręcznie.

#### Loss i optymalizator:

* **CrossEntropyLoss**
* **Adam**, `lr=0.001`

---

### Pętla treningowa

Dla każdej epoki:

#### W trybie `model.train()`:

Dla każdego batcha:

* oblicz logits (predykcję),
* policz stratę,
* wykonaj:

  * `optimizer.zero_grad()`
  * `loss.backward()`
  * `optimizer.step()`,
* policz accuracy.

#### Po każdej epoce wypisz:

* średni **train loss**
* średnią **val accuracy**

---

### Walidacja i zapis najlepszego modelu

Napisz funkcję ewaluacyjną, która:

* używa `model.eval()`,
* używa `torch.no_grad()`.

Oblicza:

* `val_loss`
* `val_accuracy`

#### Po epce:

Jeśli `val_loss` jest **najlepszy dotychczas**, zapisz model:

```python
torch.save(model.state_dict(), "best_kmnist.pth")
print("Zapisano nowy najlepszy model!")
```

---

### Testowanie

1. Załaduj najlepszy model:

   ```python
   model.load_state_dict(torch.load("best_kmnist.pth"))
   ```

2. Uruchom ewaluację na zbiorze testowym.

3. Wypisz finalne **test accuracy**.


In [ ]:
# Komórka odpowiedzialna za przygotowanie danych.
import os
import csv
from torchvision.datasets import KMNIST
import torchvision.transforms as transforms
from PIL import Image
import numpy as np

def export_kmnist(root="kmnist_data"):
    os.makedirs(root, exist_ok=True)

    for split in ["train", "test"]:
        dataset = KMNIST(
            root="./raw_kmnist",
            train=(split=="train"),
            download=True
        )

        img_dir = os.path.join(root, split)
        os.makedirs(img_dir, exist_ok=True)

        csv_path = os.path.join(root, f"{split}_labels.csv")

        with open(csv_path, "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["filename", "label"])

            for i, (img, label) in enumerate(dataset):
                filename = f"img_{i:05d}.jpg"
                img_path = os.path.join(img_dir, filename)

                # zapis do jpg
                np_img = np.array(img)
                Image.fromarray(np_img).save(img_path)

                writer.writerow([filename, label])

export_kmnist()


100%|██████████| 18.2M/18.2M [01:15<00:00, 239kB/s] 
100%|██████████| 29.5k/29.5k [00:00<00:00, 95.2kB/s]
100%|██████████| 3.04M/3.04M [00:06<00:00, 506kB/s] 
100%|██████████| 5.12k/5.12k [00:00<00:00, 1.29MB/s]


In [ ]:
# Twoje rozwiązanie

Epoch 1: train_loss=0.4521, train_acc=0.8384, val_loss=0.3256, val_acc=0.8792
Zapisano nowy najlepszy model!
Epoch 2: train_loss=0.2884, train_acc=0.8957, val_loss=0.2800, val_acc=0.9010
Zapisano nowy najlepszy model!
Epoch 3: train_loss=0.2457, train_acc=0.9102, val_loss=0.2644, val_acc=0.9030
Zapisano nowy najlepszy model!
Epoch 4: train_loss=0.2133, train_acc=0.9213, val_loss=0.2592, val_acc=0.9030
Zapisano nowy najlepszy model!
Epoch 5: train_loss=0.1864, train_acc=0.9312, val_loss=0.2325, val_acc=0.9142
Zapisano nowy najlepszy model!
Epoch 6: train_loss=0.1631, train_acc=0.9389, val_loss=0.2236, val_acc=0.9173
Zapisano nowy najlepszy model!
Epoch 7: train_loss=0.1413, train_acc=0.9476, val_loss=0.2368, val_acc=0.9163
Epoch 8: train_loss=0.1216, train_acc=0.9546, val_loss=0.2493, val_acc=0.9143
Epoch 9: train_loss=0.1041, train_acc=0.9613, val_loss=0.2641, val_acc=0.9120
Epoch 10: train_loss=0.0888, train_acc=0.9671, val_loss=0.2696, val_acc=0.9125
Test accuracy: 0.9134
